In [1]:
import os
import re
from datasketch import MinHash
from itertools import combinations
from nltk import ngrams
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [13]:
# ====== Cấu hình ======
FOLDER_PATH = "../1.CollectingDocuments/data_clean"  # Thư mục chứa các file .txt
NGRAMS = [2, 3]       # Dùng bigram và trigram
MINHASH_SIZE = 128    # Số lượng hash (kích thước vector minhash)
SIM_THRESHOLD = 0.5   # Ngưỡng tương tự để xem là trùng
ENCODING = "utf-8"

In [3]:
# ====== 1. Làm sạch text ======
def clean_text(text):
    """Chuyển chữ thường, bỏ ký tự đặc biệt, giữ lại chữ & số."""
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [4]:
# ====== 2. Sinh n-gram ======
def get_ngrams(text, n):
    """Trả về danh sách các n-gram (chuỗi n từ liên tiếp)."""
    tokens = word_tokenize(text)
    return [" ".join(gram) for gram in ngrams(tokens, n)]


In [5]:
# ====== 3. Tạo MinHash ======
def create_minhash(ngrams_set, num_perm=MINHASH_SIZE):
    """Sinh chữ ký MinHash từ tập n-gram."""
    m = MinHash(num_perm=num_perm)
    for ng in ngrams_set:
        m.update(ng.encode("utf8"))
    return m

In [6]:
# ====== 4. Đọc các file trong thư mục ======
def load_documents(folder):
    docs = {}
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            path = os.path.join(folder, filename)
            with open(path, "r", encoding=ENCODING) as f:
                text = f.read()
                docs[filename] = text
    return docs

In [7]:
# ====== 5. Tiền xử lý + sinh MinHash cho từng doc ======
def build_minhashes(docs):
    minhashes = {}
    for name, text in tqdm(docs.items(), desc="Tạo MinHash"):
        text = clean_text(text)
        ngrams_all = set()
        for n in NGRAMS:
            ngrams_all |= set(get_ngrams(text, n))
        minhashes[name] = create_minhash(ngrams_all)
    return minhashes


In [14]:
# ====== 6. So sánh cặp tài liệu ======
def compute_similarities(minhashes, threshold=SIM_THRESHOLD):
    similar_pairs = []
    for (doc1, m1), (doc2, m2) in tqdm(list(combinations(minhashes.items(), 2)), desc="So sánh"):
        sim = m1.jaccard(m2)
        if sim >= threshold:
            similar_pairs.append((doc1, doc2, round(sim, 3)))
    return similar_pairs

In [9]:
# ====== 7. Ghi kết quả ra CSV ======
def save_results(pairs, output_path="similar_docs.csv"):
    import csv
    with open(output_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Document 1", "Document 2", "Similarity"])
        for d1, d2, s in pairs:
            writer.writerow([d1, d2, s])
    print(f"✅ Kết quả đã lưu vào {output_path}")

In [ ]:
# ====== 8. Chạy toàn bộ pipeline ======
print("📂 Đang đọc file...")
docs = load_documents(FOLDER_PATH)

print(f"📄 Tổng số tài liệu: {len(docs)}")
minhashes = build_minhashes(docs)

📂 Đang đọc file...
📄 Tổng số tài liệu: 407


Tạo MinHash: 100%|██████████| 407/407 [02:42<00:00,  2.51it/s]


🔍 Đang tính độ tương tự...


So sánh: 100%|██████████| 82621/82621 [00:01<00:00, 48146.46it/s]



🧭 Các cặp tài liệu trùng hoặc giống nhau:
 - Cam_nang_du_lich_Ky_Co.txt <-> Cam_nang_du_lich_Lam_ong.txt: độ tương tự = 1.0
✅ Kết quả đã lưu vào similar_docs.csv


In [15]:
print("🔍 Đang tính độ tương tự...")
similar_pairs = compute_similarities(minhashes)

if similar_pairs:
    print("\n🧭 Các cặp tài liệu trùng hoặc giống nhau:")
    for d1, d2, s in similar_pairs:
        print(f" - {d1} <-> {d2}: độ tương tự = {s}")
    save_results(similar_pairs)
else:
    print("❌ Không có cặp tài liệu nào giống nhau vượt ngưỡng.")

🔍 Đang tính độ tương tự...


So sánh: 100%|██████████| 82621/82621 [00:01<00:00, 52337.49it/s]


🧭 Các cặp tài liệu trùng hoặc giống nhau:
 - Cam_nang_du_lich_Hung_Yen.txt <-> Cam_nang_du_lich_Ninh_Chu.txt: độ tương tự = 0.648
 - Cam_nang_du_lich_Ky_Co.txt <-> Cam_nang_du_lich_Lam_ong.txt: độ tương tự = 1.0
 - Cam_nang_du_lich_Ninh_Chu.txt <-> Cam_nang_du_lich_Thai_Binh.txt: độ tương tự = 0.523
 - Cam_nang_du_lich_Quang_Binh.txt <-> Cam_nang_du_lich_Quang_Tri.txt: độ tương tự = 0.5
 - Chiem_nguong_ve_ep_hiem_co_o_Thac_Mo_Binh_Phuoc_-_HoaBinhTourist.txt <-> Thac_Mo_Binh_Phuoc_Kham_pha_ve_ep_hiem_co_kho_tim.txt: độ tương tự = 0.539
 - Hang_En__Wikipedia_tieng_Viet.txt <-> ong_Phong_Nha__Wikipedia_tieng_Viet.txt: độ tương tự = 0.539
 - Khu_Du_Lich_Buu_Long,_Toa_o_Vui_Choi_Cuc_Hot_Gan_Sai_Gon_-_Klook_Blog.txt <-> Kinh_Nghiem_i_ia_ao_Cu_Chi,_Di_Tich_Noi_Tieng_O_Sai_Gon_-_Klook_Blog.txt: độ tương tự = 0.609
 - Thac_amb'ri__Wikipedia_tieng_Viet.txt <-> Thac_Pongour__Wikipedia_tieng_Viet.txt: độ tương tự = 0.602
 - Thac_amb'ri__Wikipedia_tieng_Viet.txt <-> Thac_Prenn__Wikipedia_tieng_Vie

In [ ]:
import os
import pandas as pd

# ==== CẤU HÌNH ====
csv_path = "similar_docs.csv"  # File CSV chứa các cặp và độ tương đồng
folder_path = "../1.CollectingDocuments/data_clean"  # Thư mục chứa các file .txt
threshold = 0.8  # Ngưỡng tương đồng

# ==== ĐỌC FILE CSV ====
df = pd.read_csv(csv_path)

# Lọc các cặp có độ tương đồng cao hơn ngưỡng
high_sim = df[df["Similarity"] > threshold]

print(f"🔍 Phát hiện {len(high_sim)} cặp có Similarity > {threshold}")

# ==== XÁC ĐỊNH DANH SÁCH FILE CẦN XÓA ====
files_to_delete = set(high_sim["Document 2"].tolist())  # chỉ xóa file thứ 2 trong mỗi cặp

print(f"📂 Sẽ xóa {len(files_to_delete)} file trùng lặp:")

for f in files_to_delete:
    print("  -", f)

# ==== XÁC NHẬN TRƯỚC KHI XÓA ====
confirm = input("\n❓Bạn có chắc chắn muốn xóa các file này không? (y/n): ").lower()
if confirm != "y":
    print("❎ Hủy thao tác.")
    exit()

# ==== XÓA FILE ====
deleted = 0
for f in files_to_delete:
    file_path = os.path.join(folder_path, f)
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            deleted += 1
        except Exception as e:
            print(f"⚠️ Không thể xóa {f}: {e}")

print(f"\n✅ Đã xóa thành công {deleted}/{len(files_to_delete)} file có độ tương đồng > {threshold}")


🔍 Phát hiện 1 cặp có Similarity > 0.8
📂 Sẽ xóa 1 file trùng lặp:
  - Cam_nang_du_lich_Lam_ong.txt
❎ Hủy thao tác.

✅ Đã xóa thành công 1/1 file có độ tương đồng > 0.8


: 